# ResNet18の転移学習
このリンクを押すと[google colab](https://colab.research.google.com/github/yisikawa/PyTorch_TL_ResNet18/blob/main/TL_ResNet18.ipynb)へ移動
### GitHub上のflower_dataをgoogle colab上にコピーして、学習する
##### 注意事項：メニュー「ランタイム」下「ランタイムのタイプを変更」を選び、「ハードウェアアクセラレータ」を「**GPU**」に設定して保存
### flower_dataは以下のフォルダ構成になっている
- flower_data
    - train　学習用
        - daisy
        -dandelion
        -roses
        -sunflowers
        -tulips
    -val　　評価用
        - daisy
        -dandelion
        -roses
        -sunflowers
        -tulips

# flower_dataをGitHubからクローン
#### [GitHub](https://github.com/yisikawa/PyTorch_TL_ResNet18.git)から学習用データをクローンし、確認する

In [ ]:
!git clone https://github.com/yisikawa/PyTorch_TL_ResNet18.git
!ls -l PyTorch_TL_ResNet18

# 必要なモジュールをインポートする

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets,models,transforms
from torch.utils.data import DataLoader

#**GPU**の使用を設定する

In [ ]:
device = "cuda" if torch.cuda.is_available else "cpu"

#画像データの前処理
#### サイズ等を変更するなどの前準備を行う

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

#flower_dataを取り込む
#### 学習用と評価用のデータをミニバッチサイズ32でシャフルした形で取り込む設定をする

In [ ]:
train_dataset = datasets.ImageFolder("./PyTorch_TL_ResNet18/flower_data/train",transform=transform)
test_dataset = datasets.ImageFolder("./PyTorch_TL_ResNet18/flower_data/val",transform=transform)
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)

#ResNet18のモデルをロードする
#### 学習済みのモデルをロードする

In [ ]:
model = models.resnet18(pretrained=True)

# モデルを転移学習用に準備する
#### 最終層以外は自動微分を無効(False)に設定し、出力を1000種類から5種類に変更する

In [ ]:
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(512,5)
model.to(device)

# GPUを使用して学習をおこなう
- ソフトマックス交差エントロピー損失関数、最適化関数(Adam)を設定
- エポック数(15)分のループ
    - ミニバッチ数(32)分のループ（データはシャッフル）
        - イメージと正解ラベルを取り出す
        - 勾配を初期化
        - 順伝播をおこなう
        - 損失率,正解率を求める
        - 誤差逆伝播を行う
        - 重みを更新する
    - エポックごとの損失率、正解率をプリントする

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr=0.001)
num_epochs = 15
losses = []
accs = []
for epochs in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0
    for imgs,labels in train_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        output = model(imgs)
        loss = criterion(output,labels)
        running_loss += loss.item()
        pred = torch.argmax(output,dim=1)
        running_acc  += torch.mean(pred.eq(labels).float())
        loss.backward()
        optimizer.step()
    running_loss /= len(train_loader)
    running_acc /= len(train_loader)
    losses.append(running_loss)
    accs.append(running_acc)
    print("epoch: {}, loss: {}, acc: {}".format(epochs,running_loss,running_acc))


# 損失率の推移を表示

In [ ]:
plt.plot(losses)

# 正解率の推移を表示

In [ ]:
plt.plot(accs)

#評価用データをランダムに抽出し、予測、正解、画像を表示

In [ ]:
test_iter = iter(test_loader)
testimgs,testlabels = test_iter.next()
testimg = testimgs[0]
testimg_permute = testimg.permute(1,2,0)
testimg_permute = 0.5*testimg_permute + 0.5
testimg_permute = np.clip(testimg_permute,0,1)
plt.imshow(testimg_permute)
flower_name = ["daisy","dandelion","rose","sunflower","tulip"]
testimgs = testimgs.to(device)
testlabels = testlabels.to(device)
output = model(testimgs)
pred = torch.argmax(output,dim=1)
print("    予測: {} >>> 正解: {}".format(flower_name[pred[0].item()],flower_name[testlabels[0].item()]))